In [1]:
from keras import backend as K
from keras.applications.vgg16 import VGG16
from keras.layers import GlobalAveragePooling2D, Flatten, Dense, Input, Dropout
from keras.layers.normalization import BatchNormalization
from keras.models import Model, load_model
from keras.optimizers import Adam, RMSprop, SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2


file = 'all'

if file=='all':
    path = '/home/ubuntu/nbs/state_farm/data/'
    batch_size=64
    nb_train_samples = 34000
    nb_validation_samples = 4800
elif file=='sample':
    path = '/home/ubuntu/nbs/state_farm/data/sample/'
    batch_size=64
    nb_train_samples = 1600
    nb_validation_samples = 300
elif file=='mini_sample':
    path = '/home/ubuntu/nbs/state_farm/data/mini_sample/'
    batch_size=1
    nb_train_samples = 200
    nb_validation_samples = 20
else:
    print('Unknown file name')
    exit()

#path = '/home/ubuntu/nbs/state_farm/data/'
target_size=(224, 224)

#generate the batches
def get_batches(directory, target_size=target_size, batch_size=batch_size, shuffle=False):
    datagen = ImageDataGenerator()
    return datagen.flow_from_directory(directory=directory,
                                          target_size=target_size,
                                          batch_size=batch_size,
                                          class_mode='categorical',
                                          shuffle=shuffle)

train_gen = ImageDataGenerator(rotation_range=0, 
                                     width_shift_range=0, 
                                     height_shift_range=0.1, 
                                     shear_range=0.1, 
                                     zoom_range=0.2, 
                                     channel_shift_range=10,
                                     fill_mode='nearest')

batches = train_gen.flow_from_directory(directory=path+'train_set',  
                                        target_size=target_size,
                                        batch_size=batch_size,
                                        class_mode='categorical',
                                        shuffle=True)

#batches = get_batches(path+'train_set', shuffle=True)

valid_batches = get_batches(path+'valid_set', batch_size=batch_size*2, shuffle=False)



Using Theano backend.
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)


Found 800 images belonging to 10 classes.
Found 300 images belonging to 10 classes.


In [2]:
# initialize the model
initial_model = VGG16(weights="imagenet", include_top = True)

In [3]:
# Store the fully connected layers
fc1 = initial_model.layers[-3]
fc2 = initial_model.layers[-2]
preds = Dense(batches.num_class, activation='softmax')
#(initial_model.layers[-2].output)

# Create the dropout layers
dropout1 = Dropout(0.5)
dropout2 = Dropout(0.5)

# Reconnect the layers
x = dropout1(fc1.output)
x = fc2(x)
x = dropout2(x)
predictors = preds(x)

# Create a new model
model = Model(input=initial_model.input, output=predictors)

# we freeze the other layers 
for layer in initial_model.layers: layer.trainable=False

opt = Adam(lr=0.001)

model.compile(optimizer=opt,
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

model.fit_generator(batches, epochs=10, 
                    steps_per_epoch=nb_train_samples // batch_size,
                    validation_data=valid_batches, 
                    validation_steps=nb_validation_samples // batch_size)

/home/ubuntu/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:18: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Softmax.0, inputs=/input_1)`


Epoch 1/10
25/25 [==============================] - 43s - loss: 4.6938 - acc: 0.1481 - val_loss: 2.5756 - val_acc: 0.1776
Epoch 2/10
25/25 [==============================] - 40s - loss: 3.6527 - acc: 0.2312 - val_loss: 2.3602 - val_acc: 0.2791
Epoch 3/10
25/25 [==============================] - 42s - loss: 2.8344 - acc: 0.3451 - val_loss: 2.3447 - val_acc: 0.2664
Epoch 4/10
25/25 [==============================] - 42s - loss: 2.7031 - acc: 0.3650 - val_loss: 2.1185 - val_acc: 0.3318
Epoch 5/10
25/25 [==============================] - 40s - loss: 2.4197 - acc: 0.4014 - val_loss: 2.3045 - val_acc: 0.3227
Epoch 6/10
25/25 [==============================] - 40s - loss: 2.3830 - acc: 0.3982 - val_loss: 2.4157 - val_acc: 0.3198
Epoch 7/10
25/25 [==============================] - 42s - loss: 2.1379 - acc: 0.4467 - val_loss: 2.1614 - val_acc: 0.3411
Epoch 8/10
25/25 [==============================] - 42s - loss: 2.1028 - acc: 0.4713 - val_loss: 2.2832 - val_acc: 0.3668
Epoch 9/10
25/25 [======

In [4]:
for layer in model.layers[:10]: 
    layer.trainable = False 

for layer in model.layers[10:]: 
    layer.trainable = True
    
opt = SGD(lr=10e-5) 
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
model.fit_generator(batches, epochs=15, 
                    steps_per_epoch=nb_train_samples // batch_size,
                    validation_data=valid_batches, 
                    validation_steps=nb_validation_samples // batch_size)

Epoch 1/15
25/25 [==============================] - 72s - loss: 1.4261 - acc: 0.5606 - val_loss: 1.8854 - val_acc: 0.3738
Epoch 2/15
25/25 [==============================] - 70s - loss: 1.1839 - acc: 0.6093 - val_loss: 1.8421 - val_acc: 0.4042
Epoch 3/15
25/25 [==============================] - 70s - loss: 1.0733 - acc: 0.6425 - val_loss: 1.6881 - val_acc: 0.4907
Epoch 4/15
25/25 [==============================] - 68s - loss: 1.0004 - acc: 0.6676 - val_loss: 1.7314 - val_acc: 0.4360
Epoch 5/15
25/25 [==============================] - 70s - loss: 0.9760 - acc: 0.6657 - val_loss: 1.6930 - val_acc: 0.4673
Epoch 6/15
25/25 [==============================] - 100s - loss: 0.8235 - acc: 0.7250 - val_loss: 1.6744 - val_acc: 0.4797
Epoch 7/15
25/25 [==============================] - 141s - loss: 0.7745 - acc: 0.7400 - val_loss: 1.5395 - val_acc: 0.4907
Epoch 8/15
25/25 [==============================] - 158s - loss: 0.7545 - acc: 0.7462 - val_loss: 1.5797 - val_acc: 0.4942
Epoch 9/15
24/25 [===

MemoryError: Error allocating 1644167168 bytes of device memory (out of memory).
Apply node that caused the error: GpuAllocEmpty(Assert{msg='The convolution would produce an invalid shape (dim[0] < 0).'}.0, Assert{msg='The convolution would produce an invalid shape (dim[1] < 0).'}.0, Assert{msg='The convolution would produce an invalid shape (dim[2] <= 0).'}.0, Assert{msg='The convolution would produce an invalid shape (dim[3] <= 0).'}.0)
Toposort index: 273
Inputs types: [TensorType(int64, scalar), TensorType(int64, scalar), TensorType(int64, scalar), TensorType(int64, scalar)]
Inputs shapes: [(), (), (), ()]
Inputs strides: [(), (), (), ()]
Inputs values: [array(128), array(64), array(224), array(224)]
Outputs clients: [[GpuDnnConv{algo='small', inplace=True}(GpuContiguous.0, GpuContiguous.0, GpuAllocEmpty.0, GpuDnnConvDesc{border_mode='half', subsample=(1, 1), conv_mode='conv', precision='float32'}.0, Constant{1.0}, Constant{0.0})]]

HINT: Re-running with most Theano optimization di

In [ ]:
# predictions on new data
test_batches = get_batches(path+'test', batch_size=batch_size)
preds = model.predict_generator(test_batches, steps=test_batches.samples//batch_size)


In [ ]:
a=1